In [ ]:
using System.IO;
using System.Text.Json;

In [ ]:
public class JsonColumn {
    public string Name { get; set; }
    public string Path { get; set; }
    public string DataType { get; set; }
    public JsonValueKind ValueKind { get; set; }

}

In [ ]:
// var fileName = "emptyarray.json";
var fileName = "transactions.json";
var jsonString = File.ReadAllText(fileName);
// var jsonObject = JsonSerializer.Deserialize<Dictionary<string, object>>(jsonString);
jsonString

{
    "results": [
      {
        "id": "062b3a9b-b04f-4d9c-9ebc-81dcb5bbf285",
        "transactionId": "356deb9f-98e6-4d6f-8db3-b378241895a1",
        "note": "Direct income 38e8c917-0afc-4583-8ffe-1b53ad601197",
        "bankAccountRef": {
          "id": "ff8fdc4c-2aad-4ec1-9430-6bd201832eb3",
          "name": "Undeposited Funds"
        },
        "date": "2022-04-19T00:00:00",
        "status": "Unreconciled",
        "currency": "GBP",
        "currencyRate": 1,
        "lines": [
          {                                                                                                                                                                                                                                                                                                                                                            
            "description": "Payment for direct income $38e8c917-0afc-4583-8ffe-1b53ad601197",
            "recordRef": {
              "id": "38e

In [ ]:
JsonDocument document = JsonDocument.Parse(jsonString);

In [ ]:
JsonElement rootElement = document.RootElement;
rootElement.ValueKind

Object

In [ ]:
// rootElement.GetType()

In [ ]:
// var rootChildren = rootElement.EnumerateObject();
// foreach (var child in rootChildren)
// {
//     Console.WriteLine(child.Name+" : "+child.Value.ValueKind);
// }

In [ ]:
// var columns = new List<JsonColumn>();
// var rootColumn = new JsonColumn
// {
//     Name = "",
//     Path = "$",
//     ValueKind = rootElement.ValueKind,
//     DataType = $"root{rootElement.ValueKind}"
// };
// columns.Add(rootColumn);
// var rootChildren = rootElement.EnumerateObject();
// foreach (var child in rootChildren)
// {
//     ProcessJsonProperty(child, rootColumn.Path, columns);
// };
// columns

In [ ]:
public IList<JsonColumn> ProcessJsonDocument(JsonDocument document)
{
    var columns = new List<JsonColumn>();
    var rootColumn = new JsonColumn
    {
        Name = "$",
        Path = "$",
        ValueKind = document.RootElement.ValueKind,
        DataType = $"root{document.RootElement.ValueKind}"
    };
    columns.Add(rootColumn);
    switch (document.RootElement.ValueKind)
    {
        case JsonValueKind.Object:
            var rootChildren = document.RootElement.EnumerateObject();
            foreach (var child in rootChildren)
            {
                ProcessJsonProperty(child, rootColumn.Path, columns);
            };
            break;
        case JsonValueKind.Array:
            rootColumn.Path = "$[]";
            ProcessJsonArray(document.RootElement, rootColumn, columns);
            break;
        default:
            throw new ArgumentException("Unsupported JSON value kind", nameof(document.RootElement));
    }
    
    return columns;
}

public void ProcessJsonProperty(JsonProperty jsonProperty, string parent, List<JsonColumn> columns)
{
    var column = new JsonColumn
    {
        Name = jsonProperty.Name,
        Path = $"{parent}.['{jsonProperty.Name}']",
        ValueKind = jsonProperty.Value.ValueKind
    };

    columns.Add(column);

    switch (jsonProperty.Value.ValueKind)
    {
        case JsonValueKind.String:
            // column.DataType = typeof(string).ToString();
            column.DataType = "string";
            break;
        case JsonValueKind.Number:
            column.DataType = "double";
            // column.DataType = typeof(double).ToString();
            break;
        case JsonValueKind.True:
        case JsonValueKind.False:
            column.DataType = "boolean";
            // column.DataType = typeof(bool).ToString();
            break;
        case JsonValueKind.Null:
        break;
        case JsonValueKind.Object:
            column.DataType = $"{jsonProperty.Name}Object";
            foreach (var child in jsonProperty.Value.EnumerateObject())
            {
                ProcessJsonProperty(child, column.Path, columns);
            }
            break;
        case JsonValueKind.Array:
            ProcessJsonArray(jsonProperty.Value, column, columns);
            break;
        default:
            throw new NotImplementedException("Unsupported JSON value kind: " + jsonProperty.Name);
    }
}

public void ProcessJsonArray(JsonElement jsonArray, JsonColumn parent, List<JsonColumn> columns) 
{
    var elements = jsonArray.EnumerateArray();
    if (!IsSameJsonValueKind(elements))
    {
        throw new ArgumentException("All elements in the array must have the same JSON value kind");
    }
    else if (!elements.Any()) 
    {
        parent.DataType = $"EmptyArray";
    }
    else 
    {
        var first = elements.First();
        parent.DataType = $"{first.ValueKind}Array";
        switch (first.ValueKind) 
        {
            case JsonValueKind.String:
            case JsonValueKind.Number:
                parent.DataType = $"{first.ValueKind}Array";
                break;
            case JsonValueKind.True:
            case JsonValueKind.False:
                parent.DataType = "BooleanArray";
                break;
            case JsonValueKind.Null:
                parent.DataType = "NullArray";
                break;
            case JsonValueKind.Array:
                parent.DataType = "ArrayArray";
                break;
            case JsonValueKind.Object:
                parent.DataType = "ObjectArray";
                foreach (var prop in first.EnumerateObject())
                {
                    ProcessJsonProperty(prop, $"{parent.Path}[]", columns);
                }
                break;
            default:
                throw new ArgumentException("Unsupported JSON value kind");
        }
        
    }
}

public bool IsSameJsonValueKind(IEnumerable<JsonElement> elements)
{
    if (elements.Count() <= 1) return true;
    else return elements.All(x => x.ValueKind == elements.First().ValueKind);
}


In [ ]:
var rs = rootElement.GetProperty("results");
var childArr = rs.EnumerateArray();
var obj = childArr.All(x => x.ValueKind == JsonValueKind.Object);
// foreach (var item in childArr)
// {
//     var props = item.EnumerateObject();
//     foreach (var prop in props)
//     {
//         Console.WriteLine(prop.Name + ": " + prop.Value.ValueKind);
//     }
// }
obj

True

In [ ]:
var rsc = ProcessJsonDocument(document);
// foreach (var row in rsc)
// {
//     Console.WriteLine(row.Path);
//     Console.WriteLine(row.Name);
//     Console.WriteLine(row.DataType);
//     Console.WriteLine(row.ValueKind);
//     Console.WriteLine("=============================");
// }

// var checkedoption = rsc.Select(x=> new KeyValuePair<string, string>(x.Name, x.Path));
// foreach (var item in checkedoption)
// {
//     Console.WriteLine(item);
// }

// Dictionary<string, string> checkedoption = rsc.Select(x => new KeyValuePair<string, string>(x.Path, x.Path)).ToDictionary(kvp => kvp.Key, kvp => kvp.Value);
// Dictionary<string, string> checkedoption = rsc.ToDictionary(kvp => kvp.Path, kvp => kvp.Path);

var checkedoption = rsc.Where(x=> !x.Path.StartsWith("$.['results'][]") 
                                && (int)x.ValueKind < 7
                                && (int)x.ValueKind > 2);

File.WriteAllText("checkedoptionsv2.json", JsonSerializer.Serialize(checkedoption, new JsonSerializerOptions { WriteIndented = true }));
